In [2]:
using DataFrames
using JuMP, Gurobi
using Gadfly

    .+(AbstractArray{S<:Union(GenericNormExpr{2,Float64,Variable},GenericAffExpr{CoefType,VarType},GenericNorm{P,Float64,Variable},GenericQuadExpr{Float64,Variable},AbstractJuMPScalar),N},AbstractArray{T<:Number,N}) at C:\Users\huntington\.julia\v0.3\JuMP\src\operators.jl:565
is ambiguous with: 
    .+(Union(PooledDataArray{T,R<:Integer,N},DataArray{T,N}),AbstractArray{T,N}...) at C:\Users\huntington\.julia\v0.3\DataArrays\src\broadcast.jl:284.
To fix, define 
    .+(Union(DataArray{S<:Union(GenericNormExpr{2,Float64,Variable},GenericAffExpr{CoefType,VarType},GenericNorm{P,Float64,Variable},GenericQuadExpr{Float64,Variable},AbstractJuMPScalar),N},PooledDataArray{S<:Union(GenericNormExpr{2,Float64,Variable},GenericAffExpr{CoefType,VarType},GenericNorm{P,Float64,Variable},GenericQuadExpr{Float64,Variable},AbstractJuMPScalar),R<:Integer,N}),AbstractArray{T<:Number,N})
before the new definition.
    .+(AbstractArray{S<:Union(GenericNormExpr{2,Float64,Variable},GenericAffExpr{CoefType,VarTy

In [44]:
m = Model(solver=GurobiSolver())


iGenLoad = readtable("BatteryModel_GenLoad.csv",header=true)
iPrices = readtable("BatteryModel_Prices.csv",header=true)
#iParameters = readtable("BatteryModel_Parameters.csv",header=true)
#iScheduledLoadsData = readtable("SchedulableLoads_Parameters.csv", header=true) 
iThermalParameters = readtable("ComplexThermal_Coeff.csv", header=true) 
iThermalFactors = readtable("ComplexThermal_Factors.csv", header=true) 

T = length(iPrices[:Hour]) 
pHours = iPrices[:Hour][1:T]

pBuyEnergy = iPrices[:pBuy_Energy][1:T]
pSellEnergy = iPrices[:pSell_Energy][1:T]
pNonControllableLoad = iGenLoad[:pNonControllableLoad][1:T]

pCf = iThermalParameters[:value]
pTz = iThermalFactors[:Tz][1:T]
pTx = iThermalFactors[:Tx][1:T]
pQrad = iThermalFactors[:Qrad][1:T]
pQconv = iThermalFactors[:Qconv][1:T]
pQwind = iThermalFactors[:Qwindow_solar][1:T]
pQsurf = iThermalFactors[:Qsurface_solar][1:T]
pTground = iThermalFactors[:Tground][1:T]
pCOP = 3

n=2 #thermal model order
k=6 #number of exogenous variables

6

In [61]:
pBuyEnergy;

In [51]:
##Noras Thermal Model:
#cCV= [((n+1)*k)+n] #constant coefficients - load from first tab in nora's spreadsheet
#cBTM = [T+n, k+1] #weather and build input data

@defVar(m, vTempInt[t=1:T])
@defVar(m, vPowerHVAC[t=1:T])
@defExpr(vQhvac[t=1:T], pCOP*vPowerHVAC[t])

# determine temp for t = 1 to n, from lag variables
for t=1:n 
    @addConstraint(m, pCf[1]*pTz[t+1]+pCf[2]*pTz[t]                                       #past zonal temps
        +pCf[3]*pTx[t+2]+pCf[4]*pTx[t+1]+pCf[5]*pTx[t]                      #current and past ambient temps
        +pCf[6]*pQrad[t+2]+pCf[7]*pQrad[t+1]+pCf[8]*pQrad[t]                #current and past Q radiative
        +pCf[9]*pQconv[t+2]+pCf[10]*pQconv[t+1]+pCf[11]*pQconv[t]           #current and past Q convective
        +pCf[12]*pQwind[t+2]+pCf[13]*pQwind[t+1]+pCf[14]*pQwind[t]          #current and past Q window solar
        +pCf[15]*pQsurf[t+2]+pCf[16]*pQsurf[t+1]+pCf[17]*pQsurf[t]          #current and past Q surface solar
        +pCf[18]*pTground[t+2]+pCf[19]*pTground[t+1]+pCf[20]*pTground[t] == vTempInt[t])   #current and past ground temp
end

# determine temp for t > 1 from existing temp, hvac, and lag variables
for t=(n+1):T
    @addConstraint(m, pCf[1]*vTempInt[t-1]+pCf[2]*vTempInt[t-2]                         #past zonal temps
        +pCf[3]*pTx[t]+pCf[4]*pTx[t-1]+pCf[5]*pTx[t-2]                      #current and past ambient temps
        +pCf[6]*pQrad[t]+pCf[7]*pQrad[t-1]+pCf[8]*pQrad[t-2]                #current and past Q radiative
        +pCf[9]*(pQconv[t]-vQhvac[t])+pCf[10]*(pQconv[t-1]-vQhvac[t-1])+pCf[11]*(pQconv[t-2]-vQhvac[t-2])           #current and past Q convective
        +pCf[12]*pQwind[t]+pCf[13]*pQwind[t-1]+pCf[14]*pQwind[t-2]          #current and past Q window solar
        +pCf[15]*pQsurf[t]+pCf[16]*pQsurf[t-1]+pCf[17]*pQsurf[t-2]          #current and past Q surface solar
        +pCf[18]*pTground[t]+pCf[19]*pTground[t-1]+pCf[20]*pTground[t-2] ==vTempInt[t])   #current and past ground temp
end



In [56]:
@defExpr(TotalCost, sum{pBuyEnergy[t]*vPowerHVAC[t]/1000, t=1:T})

0.0003 vPowerHVAC[1] + 0.0003 vPowerHVAC[2] + 0.0003 vPowerHVAC[3] + 0.0003 vPowerHVAC[4] + 0.0003 vPowerHVAC[5] + 0.0003 vPowerHVAC[6] + 0.0003 vPowerHVAC[7] + 0.0003 vPowerHVAC[8] + 0.0003 vPowerHVAC[9] + 0.0003 vPowerHVAC[10] + 0.0003 vPowerHVAC[11] + 0.0003 vPowerHVAC[12] + 0.0003 vPowerHVAC[13] + 0.0003 vPowerHVAC[14] + 0.0003 vPowerHVAC[15] + 0.0003 vPowerHVAC[16] + 0.0003 vPowerHVAC[17] + 0.0003 vPowerHVAC[18] + 0.0003 vPowerHVAC[19] + 0.0003 vPowerHVAC[20] + 0.0003 vPowerHVAC[21] + 0.0003 vPowerHVAC[22] + 0.0003 vPowerHVAC[23] + 0.0003 vPowerHVAC[24]

In [57]:
@setObjective(m, Min, TotalCost)

0.0003 vPowerHVAC[1] + 0.0003 vPowerHVAC[2] + 0.0003 vPowerHVAC[3] + 0.0003 vPowerHVAC[4] + 0.0003 vPowerHVAC[5] + 0.0003 vPowerHVAC[6] + 0.0003 vPowerHVAC[7] + 0.0003 vPowerHVAC[8] + 0.0003 vPowerHVAC[9] + 0.0003 vPowerHVAC[10] + 0.0003 vPowerHVAC[11] + 0.0003 vPowerHVAC[12] + 0.0003 vPowerHVAC[13] + 0.0003 vPowerHVAC[14] + 0.0003 vPowerHVAC[15] + 0.0003 vPowerHVAC[16] + 0.0003 vPowerHVAC[17] + 0.0003 vPowerHVAC[18] + 0.0003 vPowerHVAC[19] + 0.0003 vPowerHVAC[20] + 0.0003 vPowerHVAC[21] + 0.0003 vPowerHVAC[22] + 0.0003 vPowerHVAC[23] + 0.0003 vPowerHVAC[24]

In [58]:
# Print outputs
status = solve(m)
status
println("Objective value: ", getObjectiveValue(m))

Optimize a model with 48 rows, 96 columns and 268 nonzeros
Coefficient statistics:
  Matrix range    [2e-05, 1e+00]
  Objective range [3e-04, 3e-04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [4e-01, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 24 iterations and 0.00 seconds
Unbounded model


Objective value: NaN


In [73]:
#import os
ENV["PATH"]=";C:\Program Files\Git\\bin"


";C:Program FilesGit\\bin"

In [72]:
%%bash
git status

LoadError: bash not defined
while loading In[72], in expression starting on line 1